In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score
import seaborn as sns
import statsmodels.api as sm


In [2]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
df = pd.read_csv(url, sep=";")
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [5]:
df = df.dropna()
df[['volatile acidity', 'quality']].groupby('quality').mean()

,volatile acidity
quality,
3,0.884500
4,0.693962
5,0.577041
6,0.497484
7,0.403920
8,0.423333


In [7]:
bins = [0, 1.9, 2.2, 2.6, 15.5]
pd.cut(df['residual sugar'], bins=bins).value_counts()

count    1599.000000
mean        2.538806
std         1.409928
min         0.900000
25%         1.900000
50%         2.200000
75%         2.600000
max        15.500000
Name: residual sugar, dtype: float64

In [8]:
test_gre = pd.cut(x = df['residual sugar'],
                  bins = [0, 1.9, 2.2, 2.6, 15.5],
                  labels = ['bad', 'average', 'average +', 'good'])

pd.crosstab(df['quality'], test_gre)

residual sugar,bad,average,average +,good
quality,,,,
3,3,4,0,3
4,14,20,5,14
5,209,164,157,151
6,185,179,146,128
7,46,49,49,55
8,7,3,4,4


In [9]:
test_gre = pd.cut(x = df['residual sugar'],
                  bins = [200, 450, 550, 620, 800],
                  labels = ['bad', 'average', 'average +', 'good'])

pd.crosstab(df['quality'], test_gre)

residual sugar
quality
